In [3]:
# === PhonePe Pulse: unified loader + multi-select CLI (single cell) ===

from __future__ import annotations
import json
from pathlib import Path
from typing import List, Dict, Optional, Iterable
import pandas as pd

# ---------- CONFIG ----------
DATA_ROOT = Path("data")
IGNORE_META = {"success", "code", "responseTimestamp", "from", "to"}
SECTION_CHOICES = ["aggregated", "map", "top"]
TYPE_CHOICES = ["transaction", "user", "insurance"]

# ---------- IO & PATH HELPERS ----------
def _read_json(p: Path) -> Optional[dict]:
    try:
        with p.open("r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return None

def _to_float(x):
    if isinstance(x, (int, float)): return float(x)
    if isinstance(x, str):
        try: return float(x.replace(",", ""))
        except: return None
    return None

def _quarter_from_name(p: Path) -> Optional[int]:
    try:
        q = int(p.stem)
        return q if q in (1,2,3,4) else None
    except: return None

def _year_from_parent(p: Path) -> Optional[int]:
    try: return int(p.parent.name)
    except: return None

def _path_meta(p: Path) -> Dict[str, Optional[str]]:
    """
    Examples:
      data/aggregated/transaction/country/india/2019/1.json
      data/aggregated/transaction/state/karnataka/2021/4.json
      data/map/user/hover/country/india/2021/1.json
      data/top/transaction/country/india/2021/4.json
    """
    parts = p.parts
    if "data" not in parts: return {}
    i = parts.index("data")
    comps = parts[i:]
    if len(comps) < 6: return {}

    d = {"section": comps[1], "dtype": comps[2], "map_kind": None}
    j = 3
    if d["section"] == "map" and comps[j] == "hover":
        d["map_kind"] = "hover"
        j += 1
    d["geo_level"] = comps[j]; j += 1                # country | state
    d["geo_name"]  = comps[j]; j += 1                # india | state-name
    try:
        d["year"] = int(comps[j]); d["quarter"] = int(Path(comps[j+1]).stem)
    except Exception:
        d["year"]    = _year_from_parent(p)
        d["quarter"] = _quarter_from_name(p)
    return d

# ---------- PARSERS (return list[dict]) ----------
def _parse_agg_txn(payload: dict) -> List[dict]:
    rows = []
    for it in (payload.get("data", {}).get("transactionData") or []):
        cat = it.get("name")
        for pi in (it.get("paymentInstruments") or []):
            if pi.get("type") == "TOTAL":
                rows.append({"metric":"transactions","category":cat,
                             "count":_to_float(pi.get("count")),
                             "amount":_to_float(pi.get("amount"))})
    return rows

def _parse_agg_ins(payload: dict) -> List[dict]:
    rows = []
    for it in (payload.get("data", {}).get("transactionData") or []):
        for pi in (it.get("paymentInstruments") or []):
            if pi.get("type") == "TOTAL":
                rows.append({"metric":"insurance","category":it.get("name","Insurance"),
                             "count":_to_float(pi.get("count")),
                             "amount":_to_float(pi.get("amount"))})
    return rows

def _parse_agg_user(payload: dict) -> List[dict]:
    rows = []
    data = payload.get("data", {})
    agg  = data.get("aggregated") or {}
    if agg:
        rows.append({"metric":"users_summary",
                     "registeredUsers":_to_float(agg.get("registeredUsers")),
                     "appOpens":_to_float(agg.get("appOpens")),
                     "brand":None,"brand_count":None,"brand_pct":None})
    for d in (data.get("usersByDevice") or []):
        rows.append({"metric":"users_by_device",
                     "registeredUsers":None,"appOpens":None,
                     "brand":d.get("brand"),
                     "brand_count":_to_float(d.get("count")),
                     "brand_pct":_to_float(d.get("percentage"))})
    return rows

def _parse_map_txn_ins(payload: dict) -> List[dict]:
    rows = []
    for item in (payload.get("data", {}).get("hoverDataList") or []):
        name = item.get("name")
        for m in (item.get("metric") or []):
            if m.get("type") == "TOTAL":
                rows.append({"name":name,
                             "count":_to_float(m.get("count")),
                             "amount":_to_float(m.get("amount"))})
    return rows

def _parse_map_user(payload: dict) -> List[dict]:
    rows = []
    for name, vals in (payload.get("data", {}).get("hoverData") or {}).items():
        rows.append({"name":name,
                     "registeredUsers":_to_float(vals.get("registeredUsers")),
                     "appOpens":_to_float(vals.get("appOpens"))})
    return rows

def _parse_top_txn_ins(payload: dict) -> List[dict]:
    rows = []
    data = payload.get("data", {}) or {}
    for level in ("states","districts","pincodes"):
        for item in (data.get(level) or []):
            m = item.get("metric") or {}
            if m.get("type") == "TOTAL":
                rows.append({"level":level[:-1],
                             "name":item.get("entityName"),
                             "count":_to_float(m.get("count")),
                             "amount":_to_float(m.get("amount"))})
    return rows

def _parse_top_user(payload: dict) -> List[dict]:
    rows = []
    data = payload.get("data", {}) or {}
    def add(level):
        for it in (data.get(level) or []):
            rows.append({"level":level[:-1],
                         "name":it.get("name"),
                         "registeredUsers":_to_float(it.get("registeredUsers"))})
    add("states"); add("districts"); add("pincodes")
    return rows

# ---------- NORMALIZATION ----------
def _normalize_file(p: Path) -> List[dict]:
    meta = _path_meta(p)
    if not meta: return []
    payload = _read_json(p)
    if not isinstance(payload, dict): return []
    for k in list(payload.keys()):
        if k in IGNORE_META:
            payload.pop(k, None)

    section, dtype = meta["section"], meta["dtype"]
    rows: List[dict] = []
    try:
        if section == "aggregated":
            if dtype == "transaction": rows = _parse_agg_txn(payload)
            elif dtype == "insurance": rows = _parse_agg_ins(payload)
            elif dtype == "user":      rows = _parse_agg_user(payload)
        elif section == "map":
            if dtype in ("transaction","insurance"): rows = _parse_map_txn_ins(payload)
            elif dtype == "user":                    rows = _parse_map_user(payload)
        elif section == "top":
            if dtype in ("transaction","insurance"): rows = _parse_top_txn_ins(payload)
            elif dtype == "user":                    rows = _parse_top_user(payload)
    except Exception:
        rows = []

    for r in rows:
        r.update(meta)
    return rows

def load_all_rows() -> pd.DataFrame:
    files = sorted(DATA_ROOT.rglob("*.json"))
    out: List[dict] = []
    for p in files:
        q = _quarter_from_name(p); y = _year_from_parent(p)
        if q is None or y is None: continue
        out.extend(_normalize_file(p))
    df = pd.DataFrame(out)
    if not df.empty:
        df["period"] = pd.PeriodIndex(
            df["year"].astype("Int64").astype(str) + "Q" + df["quarter"].astype("Int64").astype(str),
            freq="Q")
        df["geo"] = df["geo_level"].str.cat(df["geo_name"], sep=":")
        df["section_type"] = df["section"].str.cat(df["dtype"], sep="/")
    return df

# ---------- CLI UTILS ----------
def _pick(prompt: str, options: List[str]) -> str:
    print(f"\n{prompt}")
    for i,o in enumerate(options,1):
        print(f"{i}. {o}")
    while True:
        s = input("Choose number: ").strip()
        try:
            k = int(s); 
            if 1 <= k <= len(options):
                return options[k-1]
        except: pass
        print("Invalid. Try again.")

def _pick_multi(prompt: str, options: List[str], allow_all=True) -> List[str]:
    print(f"\n{prompt} (comma-separated indexes{' or all' if allow_all else ''})")
    for i,o in enumerate(options,1):
        print(f"{i}. {o}")
    while True:
        s = input("Choose: ").strip().lower()
        if allow_all and s in ("all","*"): return options
        try:
            idxs = [int(x) for x in s.replace(" ","").split(",") if x]
            picked = [options[i-1] for i in idxs if 1 <= i <= len(options)]
            if picked:
                seen=set(); out=[]
                for x in picked:
                    if x not in seen: out.append(x); seen.add(x)
                return out
        except: pass
        print("Invalid. Try again.")

def _parse_years(user_in: str, available: Iterable[int]) -> List[int]:
    avail = sorted(set(int(x) for x in available))
    t = user_in.strip().lower()
    if t in ("all","*"): return avail
    years=set()
    for part in t.replace(" ","").split(","):
        if "-" in part:
            a,b = part.split("-",1)
            try:
                a,b=int(a),int(b)
                for y in avail:
                    if a<=y<=b: years.add(y)
            except: pass
        else:
            try:
                y=int(part)
                if y in avail: years.add(y)
            except: pass
    return sorted(years) or avail

def _parse_quarters(user_in: str) -> List[int]:
    t = user_in.strip().lower()
    if t in ("all","*"): return [1,2,3,4]
    qs=set()
    for part in t.replace(" ","").split(","):
        try:
            q=int(part)
            if q in (1,2,3,4): qs.add(q)
        except: pass
    return sorted(qs) or [1,2,3,4]

# ---------- PROGRAMMATIC API ----------
def query_data(
    sections: List[str],
    types: List[str],
    geo_level: str,        # "country" or "state"
    geos: List[str],       # ["india"] or list of state names
    years: List[int],
    quarters: List[int],
) -> pd.DataFrame:
    """
    Filter the normalized dataset programmatically.
    """
    df = load_all_rows()
    if df.empty: return df

    mask = (
        df["section"].isin(sections)
        & df["dtype"].isin(types)
        & (df["geo_level"] == geo_level)
        & df["geo_name"].isin(geos)
        & df["year"].isin(years)
        & df["quarter"].isin(quarters)
    )
    out = df[mask].copy()
    sort_cols = [c for c in ["section","dtype","geo_level","geo_name","year","quarter"] if c in out.columns]
    if sort_cols:
        out = out.sort_values(sort_cols, kind="mergesort")
    return out

# ---------- INTERACTIVE CLI (multi-select) ----------
def interactive_query_cli(save_csv: bool = False, csv_name: str = "selection.csv") -> pd.DataFrame:
    df = load_all_rows()
    if df.empty:
        raise SystemExit("No JSON data found under ./data")

    sections = _pick_multi("Sections", SECTION_CHOICES)           # multiple
    types    = _pick_multi("Types", TYPE_CHOICES)                  # multiple
    geo_level = _pick("Geography level", ["country","state"])

    if geo_level == "state":
        avail_states = sorted(df[(df.section.isin(sections)) &
                                 (df.dtype.isin(types)) &
                                 (df.geo_level=="state")]["geo_name"].dropna().unique().tolist())
        if not avail_states:
            raise SystemExit("No states available for that combination.")
        states = _pick_multi("States", avail_states)
        geos = states
    else:
        geos = ["india"]

    subset = df[(df.section.isin(sections)) &
                (df.dtype.isin(types)) &
                (df.geo_level==geo_level) &
                (df.geo_name.isin(geos))].copy()

    years_avail = sorted(subset["year"].dropna().astype(int).unique().tolist())
    print(f"\nAvailable years: {years_avail}")
    years = _parse_years(input("Pick years (e.g. 'all', '2020-2022', '2019,2021'): "), years_avail)

    quarters = _parse_quarters(input("Pick quarters (e.g. 'all' or '1,3,4'): "))

    out = subset[subset["year"].isin(years) & subset["quarter"].isin(quarters)].copy()
    print(f"\nRows selected: {len(out):,}")

    if save_csv and not out.empty:
        out.to_csv(csv_name, index=False)
        print(f"Saved -> {csv_name}")
    return out

# (No auto-execution on import)


In [6]:
df_view = interactive_query()
df_view.head(20)


Section
1. aggregated
2. map
3. top

Type
1. transaction
2. user
3. insurance


In [ ]:
df_view.to_csv("selection.csv", index=False)
print("Saved selection.csv")
